In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install xmltodict

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import xmltodict

In [5]:
!unzip "/content/drive/My Drive/dataset_road" -d "/content/dataset"

Archive:  /content/drive/My Drive/dataset_road.zip
   creating: /content/dataset/content/Final_Dataset/
   creating: /content/dataset/content/Final_Dataset/test/
   creating: /content/dataset/content/Final_Dataset/test/images/
  inflating: /content/dataset/content/Final_Dataset/test/images/road177.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road776.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road235.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road708.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road748.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road109.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road752.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road468.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road365.png  
  inflating: /content/dataset/content/Final_Dataset/test/images/road784

In [6]:
class RoadSignDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.images_dir = os.path.join(self.root_dir, "images")
        self.annotations_dir = os.path.join(self.root_dir, "annotations")
        self.transform = transform
        self.image_files = [f for f in os.listdir(self.images_dir) if os.path.isfile(os.path.join(self.images_dir, f))]
        self.class_names = ["Trafficlight", "Stop", "Speedlimit", "Crosswalk"]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        annotation_path = os.path.join(self.annotations_dir, self.image_files[idx].replace('.jpg', '.xml').replace('.png', '.xml'))

        image = Image.open(img_path).convert('RGB')

        with open(annotation_path) as file:
            annotation = xmltodict.parse(file.read())

        objects = annotation['annotation']['object']
        if isinstance(objects, list):
            class_name = objects[0]['name']
        else:
            class_name = objects['name']

        class_name = class_name.capitalize()
        label = self.class_names.index(class_name)

        if self.transform:
            image = self.transform(image)

        return image, label


In [7]:
dataset_root = '/content/dataset/content/Final_Dataset'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [8]:
train_dataset = RoadSignDataset(root_dir=dataset_root, split='train', transform=transform)
test_dataset = RoadSignDataset(root_dir=dataset_root, split='test', transform=transform)
val_dataset = RoadSignDataset(root_dir=dataset_root, split='val', transform=transform)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
#ShuffleNetv2 and ResNet18
#model = models.resnet18(pretrained=False)
#model = models.shufflenet_v2_x1_0(pretrained=False)

#num_ftrs = model.fc.in_features
#model.fc = nn.Linear(num_ftrs, 4)

#MobileNetV3
model = models.mobilenet_v3_large(pretrained=False)

num_ftrs = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_ftrs, 4)



criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)

Using device: cuda:0


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [14]:
from tqdm import tqdm

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]')
    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_accuracy += calculate_accuracy(outputs, labels)
        train_bar.set_postfix(loss=(running_loss / len(train_loader)), accuracy=(running_accuracy / len(train_loader) * 100), refresh=True)

    model.eval()
    val_running_loss = 0.0
    val_running_accuracy = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            val_running_accuracy += calculate_accuracy(outputs, labels)

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {running_accuracy/len(train_loader)*100:.2f}%, Validation Loss: {val_running_loss/len(val_loader):.4f}, Validation Accuracy: {val_running_accuracy/len(val_loader)*100:.2f}%')



Epoch 1/25 [Training]: 100%|██████████| 20/20 [00:08<00:00,  2.37it/s, accuracy=72, loss=1.35]


Epoch 1/25, Training Loss: 1.3473, Training Accuracy: 71.97%, Validation Loss: 1.3296, Validation Accuracy: 79.38%


Epoch 2/25 [Training]: 100%|██████████| 20/20 [00:08<00:00,  2.38it/s, accuracy=73.1, loss=1.26]


Epoch 2/25, Training Loss: 1.2563, Training Accuracy: 73.06%, Validation Loss: 1.2634, Validation Accuracy: 79.38%


Epoch 3/25 [Training]: 100%|██████████| 20/20 [00:08<00:00,  2.47it/s, accuracy=73.1, loss=1.16]


...

Epoch 23/25, Training Loss: 0.4416, Training Accuracy: 82.69%, Validation Loss: 0.7714, Validation Accuracy: 79.38%


Epoch 24/25 [Training]: 100%|██████████| 20/20 [00:07<00:00,  2.55it/s, accuracy=83.1, loss=0.493]


Epoch 24/25, Training Loss: 0.4929, Training Accuracy: 83.06%, Validation Loss: 0.8102, Validation Accuracy: 79.38%


Epoch 25/25 [Training]: 100%|██████████| 20/20 [00:07<00:00,  2.58it/s, accuracy=77.1, loss=0.661]


Epoch 25/25, Training Loss: 0.6613, Training Accuracy: 77.06%, Validation Loss: 0.8122, Validation Accuracy: 79.38%


In [15]:
model.eval()
test_running_accuracy = 0.0
misclassified_counts = {label: 0 for label in range(4)}
total_counts = {label: 0 for label in range(4)}

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_running_accuracy += torch.sum(preds == labels).item()

        for label, pred in zip(labels, preds):

            total_counts[label.item()] += 1
            if label != pred:
                misclassified_counts[label.item()] += 1

test_accuracy = test_running_accuracy / len(test_loader.dataset) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Accuracy: 81.06%
